In [8]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Load pretrained VGG16 without top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

def extract_vgg16_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features.flatten()


58889256/58889256 [==============================] - 61s 1us/step


In [ ]:
def load_dataset_vgg(folder_path):
    X, y = [], []
    for label, class_name in enumerate(['no_acne', 'acne']):
        class_dir = os.path.join(folder_path, class_name)
        for fname in os.listdir(class_dir):
            path = os.path.join(class_dir, fname)
            try:
                feat = extract_vgg16_features(path)
                X.append(feat)
                y.append(label)
            except:
                continue
    return np.array(X), np.array(y)

data_path = #acne_dataset path
X, y = load_dataset_vgg(data_path)
print(f"Original dataset size: {X.shape}")

1/1 [==============================] - 0s 298ms/step
Original dataset size: (295, 25088)


In [10]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

svm_model = SVC(kernel='rbf', probability=True)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["No Acne", "Acne"]))


              precision    recall  f1-score   support

     No Acne       1.00      0.98      0.99        46
        Acne       0.97      1.00      0.99        34

    accuracy                           0.99        80
   macro avg       0.99      0.99      0.99        80
weighted avg       0.99      0.99      0.99        80



In [41]:
def predict_image_vgg(img_path):
    feat = extract_vgg16_features(img_path).reshape(1, -1)
    pred = svm_model.predict(feat)[0]
    prob = svm_model.predict_proba(feat)[0][pred]
    label = "Acne" if pred else "No Acne"
    
    if pred == 1 and prob >= 0.60:
        print(f"Prediction: Acne with High Confidence ({prob*100:.2f}%)")
    elif pred == 1 and prob < 0.60:
        print(f"Prediction: Slight Acne ({prob*100:.2f}%)")
    else:
        print(f"Prediction: No Acne ({prob*100:.2f}%)")


In [ ]:
predict_image_vgg(#sample input image path to predict)

1/1 [==============================] - 0s 245ms/step
Prediction: Slight Acne (51.86%)


In [45]:
import joblib

# Save the trained SVM model
joblib.dump(svm_model, 'svm_acne_vgg_model.pkl')
print("Model saved successfully as 'svm_acne_vgg_model.pkl'")

Model saved successfully as 'svm_acne_vgg_model.pkl'


In [ ]:
import joblib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# Load the pre-trained SVM model
svm_model = joblib.load('svm_acne_vgg_model.pkl')  # Your saved model

# Define the feature extraction function using VGG16
def extract_vgg16_features(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Load the VGG16 model (without top layers)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)
    
    # Extract features
    features = feature_extractor.predict(x)
    return features.flatten()  # Flatten features into a 1D vector

# Prediction function
def predict_acne(img_path):
    # Extract features from the image
    features = extract_vgg16_features(img_path).reshape(1, -1)

    # Predict using the pre-trained SVM model
    pred = svm_model.predict(features)[0]
    prob = svm_model.predict_proba(features)[0][pred]
    
    label = "Acne" if pred == 1 else "No Acne"
    confidence = prob * 100

    # Print the result
    if pred == 1 and prob >= 0.60:
        print(f"Prediction: Acne detected ({prob*100:.2f}%)")
    elif pred == 1 and prob < 0.60:
        print(f"Prediction: Slight Possibility of Acne ({prob*100:.2f}%)")
    else:
        print(f"Prediction: No Acne detected ({prob*100:.2f}%)")

# Test the prediction
predict_acne(#sample input image path to predict)


1/1 [==============================] - 1s 828ms/step
Prediction: No Acne detected (89.27%)
